In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Helper imports
import numpy as np
from sklearn.linear_model import Lasso, LassoCV, LinearRegression, MultiTaskLassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from econml.grf import CausalForest
from econml.sklearn_extensions.linear_model import StatsModelsLinearRegression
import lightgbm as lgb

from snmm import get_linear_model_reg, get_linear_multimodel_reg
from snmm import get_model_reg, get_multimodel_reg
from snmm import get_poly_model_reg, get_poly_multimodel_reg
from snmm import SNMMDynamicDML, HeteroSNMMDynamicDML
from blip import BlipSpec, SimpleHeteroBlipSpec, SimpleBlipSpec, true_param_parse
from hetero_utils import WeightedModelFinal, LinearModelFinal, Ensemble, GCV

import warnings
warnings.simplefilter('ignore')

%matplotlib inline

# Dynamic DML for Structural Nested Mean Models

In [ ]:
semi = False
train_test = True

### Real dataL Job Corps training program

In [ ]:
df = pd.read_csv('JC.csv')
df = df.rename(columns={'Unnamed: 0':'id'}).reset_index().drop('index', axis=1).set_index(['id'])
df.head()

In [ ]:
x0_cols = list(df.columns[1:29])
x1_cols = ['everwkdy1', 'earnq4', 'earnq4mis', 'pworky1', 'health12'] #list(df.columns[29:36])
t0_cols = df.columns[[36]]
t1_cols = df.columns[[37]]
y_col = df.columns[43]

In [ ]:
print(x0_cols)

In [ ]:
print(x1_cols)

In [ ]:
x0_cols_cont = ['age', 'educ', 'mwearn', 'hhsize', 'educmum', 'educdad', 'welfarechild']

In [ ]:
df[x0_cols_cont] = StandardScaler().fit_transform(df[x0_cols_cont])
df[x1_cols] = StandardScaler().fit_transform(df[x1_cols])

In [ ]:
plt.hist(df['educmum'])
plt.show()

In [ ]:
y = df[y_col].values
X = {0: df[x0_cols], 1: df[x1_cols], 'het': df[x0_cols]}
T = {0: df[t0_cols].values, 1: df[t1_cols].values}
m = 2

In [ ]:
# run this for semi-synthetic outcomes
if semi:
    y = (1 + X[0]['educ'].values) * T[0].flatten()
    y += (1 + X[0]['mwearn'].values + X[0]['educ'].values * T[0].flatten()) * T[1].flatten() 
    y += X[0]['educ'].values + X[0]['health'].values
    y += np.random.normal(0, 1, size=(y.shape[0],))
    def true_effect_fn(t, X, T):
        return (1 + X[0]['educ'].values) if t == 0 else (1 + X[0]['mwearn'].values + X[0]['educ'].values * T[0].flatten())
    true_policy_delta = np.mean(true_effect_fn(0, X, T) + true_effect_fn(1, X, T))
    true_policy = true_policy_delta + np.mean(X[0]['educ'].values + X[0]['health'].values)
    cols = ['educ', 'mwearn']
else:
    true_policy, true_policy_delta = 0, 0
    def true_effect_fn(t, X, T):
        return np.zeros(X[0].shape[0])
    cols = ['mwearn', 'mwearn']

In [ ]:
# cat = ['age', 'educ', 'educmum', 'educdad']
# X[0] = pd.get_dummies(X[0], columns=cat)
# X[0] = pd.concat([X[0], df[cat]], axis=1)
# x0_cols = list(X[0].columns)

In [ ]:
X[0]

In [ ]:
from sklearn.model_selection import train_test_split
if train_test:
    Ttest = {}
    Xtest = {}
    (y, ytest, T[0], Ttest[0], T[1], Ttest[1],
     X[0], Xtest[0], X[1], Xtest[1], X['het'], Xtest['het']) = train_test_split(y, T[0], T[1], X[0], X[1], X['het'],
                                                                                test_size=.5, random_state=123)

# Analysis

### Define Model Parameters

In [ ]:
bs = BlipSpec(heterogeneity=True, lags=True, lag_heterogeneity=True).fit(X, T)
phi = bs.phi
phi_names = bs.phi_names

def pi(t, X, T):
    return np.ones(T[t].shape)

### Estimate High-Dimensional Linear Blip Model

In [ ]:
est = SNMMDynamicDML(m=m, phi=phi, phi_names_fn=phi_names,
                     model_reg_fn=lambda X, y: get_poly_model_reg(X, y, degree=1, interaction_only=False),
                     model_final_fn=lambda: LassoCV())

In [ ]:
est.fit(X, T, y, pi)

In [ ]:
print(est.policy_value_)
print(true_policy)
print(est.policy_delta_simple_)
print(true_policy_delta)

In [ ]:
sig = {}
for t in range(m):
    print(f'Period {t} effects')
    with pd.option_context("precision", 3):
        sig[t] = np.abs(est.psi_[t]) > 0.01
        display(est.param_summary(t, coef_thr=0.01).summary_frame())

### Post Selection Inference (not unbiased): Low Dimensional Blip Model

In [ ]:
def phi_sub(t, X, T, Tt):
    return phi(t, X, T, Tt)[:, sig[t]]

def phi_names_sub(t):
    return np.array(phi_names(t))[sig[t]]

In [ ]:
est_sub = SNMMDynamicDML(m=m, phi=phi_sub, phi_names_fn=phi_names_sub,
                         model_reg_fn=lambda X, y: get_poly_model_reg(X, y, degree=1, interaction_only=False), #lambda X, y: get_model_reg(X, y, degrees=[1, 2]),
                         model_final_fn=lambda: LinearRegression(),
                         verbose=1)

In [ ]:
est_sub.fit(Xtest, Ttest, ytest, pi)

In [ ]:
print(est_sub.policy_value_)
print(true_policy)

In [ ]:
for t in range(m):
    print(f'Period {t} effects')
    with pd.option_context("precision", 3):
        display(est_sub.param_summary(t).summary_frame(alpha=0.01))

### Policy Delta compared to all zero

For simple phi, where the structural parameters don't change dependent on the target, we can do sth very simple

In [ ]:
print(est_sub.policy_delta_simple_)
print(true_policy_delta)

### For complex phi we need to re-run the estimation for base

In [ ]:
# est_sub.fit_base()

In [ ]:
# deltapi, deltapierr = est_sub.policy_delta_complex()

In [ ]:
# print(deltapi, deltapierr)

# Non-Parametric Heterogeneity


In [ ]:
# bs = SimpleBlipSpec().fit(X, T)
bs = BlipSpec(heterogeneity=False, lags=False, lag_heterogeneity=False).fit(X, T)
phi = bs.phi
phi_names = bs.phi_names

def pi(t, X, T):
    return np.ones(T[t].shape)

In [ ]:
import seaborn as sns

def plot_true_v_est(effects, X, T, cols, mask=None):
    if mask is None:
        mask = np.ones(X[0].shape[0]) > 0
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 2, 1)
    error = effects[0][mask, 0] - true_effect_fn(0, X, T).flatten()[mask]
    rmse = np.sqrt(np.mean(error**2))
    plt.title(f'period 0: rmse={rmse:.3f}')
    if semi:
        plt.scatter(X[0][cols[0]].values[mask], effects[0][mask], label='est')
        plt.scatter(X[0][cols[0]].values[mask], true_effect_fn(0, X, T)[mask], label='true')
    else:
        sns.regplot(x=X[0][cols[0]].values[mask], y=effects[0][mask])
    plt.xlabel(cols[0])
    plt.subplot(1, 2, 2)
    pred = effects[1][mask, 0]
    if effects[1].shape[1] == 2:
        pred = pred + effects[1][mask, 1] * T[0][mask].flatten() 
    error = pred - true_effect_fn(1, X, T).flatten()[mask]
    rmse = np.sqrt(np.mean(error**2))
    plt.title(f'period 1: rmse={rmse:.3f}')
    if semi:
        plt.scatter(X[0][cols[1]].values[mask], pred, label='est')
        plt.scatter(X[0][cols[1]].values[mask], true_effect_fn(1, X, T)[mask], label='true')
    else:
        eff = effects[1][mask]
        for t in range(eff.shape[1]):
            sns.regplot(x=X[0][cols[1]][mask], y=eff[:, t])
    plt.xlabel(cols[1])
    plt.show()

In [ ]:
import seaborn as sns

def importances(mdl):
    if hasattr(mdl, 'feature_importances_'):
        return pd.DataFrame({'name': X['het'].columns, 
                             'importance': mdl.feature_importances_})
    else:
        if not (len(mdl.coef_) == len(X['het'].columns) + 1):
            return None
        return pd.DataFrame({'name': X['het'].columns, 
                             'importance': mdl.coef_[1:]})

def plot_top_k_ensemble_importances(het_est, n_models):
    
    for t in range(m):
        models = het_est.models_[t].model_.models_
        weights = het_est.models_[t].model_.weights
        inds = np.argsort(weights)[::-1][:n_models]
        rows = int(np.floor(np.sqrt(n_models)))
        cols = int(np.ceil(n_models / rows))
        plt.figure(figsize=(25, 5 * rows))
        for it, (weight, mdl) in enumerate(zip(weights[inds], np.array(models)[inds])):
            impdf = importances(mdl)
            if impdf is not None:
                plt.subplot(rows, cols, it + 1)
                sns.barplot(y=impdf['name'], x=impdf['importance'])
                plt.title(mdl.__repr__()[:10] + f', weight={weight:.3f}')
        plt.suptitle(f'Period {t}')
        plt.tight_layout()
        plt.show()

#### We define multiple final heterogeneous dynamic effect models

Causal forests fit a forest for $\theta(X)$, minimizing $E[(y - \theta(X)'T)^2]$.

For uni-variate blip model feature maps, any ML model that accepts sample weights can be used, since we can re-write it as: $E[T^2 (y/T - theta(X))^2]$, turning the problem into weighted square loss minimization. We provide the `WeightedModelFinal` wrapper that performs this transformation and wraps any ML model.

For linear models, i.e. $\theta(X) = \theta'\psi(X)$, then we can re-write this as a simple linear regression problem over the cross product of the blip feature map $\phi$ and the heterogeneous effect feature map $\psi$. We provide the `LinearModelFinal` wrapper that performs this transormations and wraps any linear model and heterogeneous effect feature map function.

In [ ]:
cf_gen = lambda ms, md, mvl, fr: lambda: CausalForest(n_estimators=1000,
                                                    max_depth=md,
                                                    min_samples_leaf=ms,
                                                    min_balancedness_tol=0.45,
                                                    max_samples=fr,
                                                    inference=False,
                                                    min_var_fraction_leaf=mvl,
                                                    min_var_leaf_on_val=True,
                                                    random_state=123)
linear_gen = lambda: LinearModelFinal(StatsModelsLinearRegression(fit_intercept=False),
                                    lambda x: x)
lasso_gen = lambda: LinearModelFinal(LassoCV(fit_intercept=False, random_state=123),
                                    lambda x: x)
polylasso_gen = lambda: LinearModelFinal(LassoCV(fit_intercept=False, random_state=123),
                                         lambda x: Pipeline([('poly', PolynomialFeatures(degree=2, include_bias=False)),
                                                             ('sc', StandardScaler())]).fit_transform(x))
rf_gen = lambda ms, md: lambda: WeightedModelFinal(RandomForestRegressor(n_estimators=100,
                                                                         min_samples_leaf=ms,
                                                                         min_weight_fraction_leaf=0.01,
                                                                         max_depth=md,
                                                                         random_state=123))
lgbm_gen = lambda lr, md: lambda: WeightedModelFinal(lgb.LGBMRegressor(num_leaves=32, n_estimators=5,
                                                                       min_child_samples=100,
                                                                       learning_rate=lr,
                                                                       max_depth=md,
                                                                       min_child_weight=0.01,
                                                                       random_state=123))

We define model generators for many configurations of the hyperparams of each type of model, each entry in `model_gens` is a function that returns an un-fitted model.

In [ ]:
model_gens = [(f'cf{it}', cf_gen(ms, md, mvl, fr))
              for it, (ms, md, mvl, fr) in enumerate([(30, 3, 0.01, .8),
                                                      (30, 5, 0.01, .8)])]
model_gens += [('ols', linear_gen), ('lassocv', lasso_gen), ('2dlassocv', polylasso_gen)]
model_gens += [(f'rf{it}', rf_gen(ms, md))
               for it, (ms, md) in enumerate([(100, 3), (100, 5)])]
model_gens += [(f'lgbm{it}', lgbm_gen(lr, md))
               for it, (lr, md) in enumerate([(0.01, 1), (0.01, 3)])]

The `GCV` estimator is an estimator that supports the `fit(X, T, y)` interface and performs cross-validation and model selection among all the models in the `model_gens` list. If `ensebmle=True`, it performs soft-max ensembling. If `ensemble=False` it uses the model with the best score.

In [ ]:
gcv_gen = lambda: GCV(model_gens=model_gens, ensemble=True, beta=1000)

In [ ]:
het_est = HeteroSNMMDynamicDML(m=m, phi=phi, phi_names_fn=phi_names,
                               model_reg_fn=lambda X, y: get_poly_model_reg(X, y, degree=1, interaction_only=False), #lambda X, y: get_model_reg(X, y, degrees=[1, 2]),
                               model_final_fn=gcv_gen)

In [ ]:
het_est.fit(X, T, y, pi)

In [ ]:
# het_est.model_final_fn = gcv_gen
# het_est.fit_final()

In [ ]:
het_est.models_[0]

In [ ]:
het_est.models_[1]

In [ ]:
print(het_est.policy_value_)
print(true_policy)

In [ ]:
print(het_est.policy_delta_simple_)
print(true_policy_delta)

In [ ]:
eff = het_est.dynamic_effects(X['het'])

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree
tree = DecisionTreeRegressor(max_depth=3).fit(X['het'].values, eff[0][:, 0])
plt.figure(figsize=(15, 10))
plot_tree(tree, feature_names=X['het'].columns)
plt.show()

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree
tree = DecisionTreeRegressor(max_depth=3).fit(X['het'].values, eff[1][:, 0])
plt.figure(figsize=(15, 10))
plot_tree(tree, feature_names=X['het'].columns)
plt.show()

In [ ]:
plot_true_v_est(eff, X, T, cols)

In [ ]:
plot_top_k_ensemble_importances(het_est, 6)

We now test how each individual model would have performed as a final model if we were to use just that model

In [ ]:
if semi:
    for name, mgen in model_gens:
        print(name, mgen())
        het_est.model_final_fn = mgen
        het_est.fit_final()
        plot_true_v_est(het_est, X, T, cols)

#### Adding Heterogeneity in Lag Treatment

In [ ]:
# bs = SimpleBlipSpec().fit(X, T)
bs = BlipSpec(heterogeneity=False, lags=True, lag_heterogeneity=False).fit(X, T)
phi = bs.phi
phi_names = bs.phi_names

In [ ]:
model_gens = [(f'cf{it}', cf_gen(ms, md, mvl, fr))
              for it, (ms, md, mvl, fr) in enumerate([(30, 3, 0.01, .8),
                                                      (30, 5, 0.01, .8)])]
model_gens += [('ols', linear_gen), ('lassocv', lasso_gen), ('2dlassocv', polylasso_gen)]

In [ ]:
gcv_gen = lambda: GCV(model_gens=model_gens, ensemble=True, beta=1000)

In [ ]:
het_est2 = HeteroSNMMDynamicDML(m=m, phi=phi, phi_names_fn=phi_names,
                                model_reg_fn=lambda X, y: get_poly_model_reg(X, y, degree=1, interaction_only=False), #lambda X, y: get_model_reg(X, y, degrees=[1, 2]),
                                model_final_fn=gcv_gen)

In [ ]:
het_est2.fit(X, T, y, pi)

In [ ]:
het_est2.models_[0]

In [ ]:
het_est2.models_[1]

In [ ]:
print(het_est2.policy_value_)
print(true_policy)

In [ ]:
print(het_est2.policy_delta_simple_)
print(true_policy_delta)

In [ ]:
eff2 = het_est2.dynamic_effects(X['het'])

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree
tree = DecisionTreeRegressor(max_depth=3).fit(X['het'].values, eff2[0][:, 0])
plt.figure(figsize=(15, 10))
plot_tree(tree, feature_names=X['het'].columns)
plt.show()

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree
tree = DecisionTreeRegressor(max_depth=3).fit(X['het'].values, eff2[1][:, 0])
plt.figure(figsize=(15, 10))
plot_tree(tree, feature_names=X['het'].columns)
plt.show()

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree
tree = DecisionTreeRegressor(max_depth=3).fit(X['het'].values, eff2[1][:, 1])
plt.figure(figsize=(15, 10))
plot_tree(tree, feature_names=X['het'].columns)
plt.show()

In [ ]:
plot_true_v_est(eff2, X, T, cols, mask=(T[0].flatten()==1))

In [ ]:
plot_true_v_est(eff2, X, T, cols, mask=(T[0].flatten()==0)) 

In [ ]:
plot_true_v_est(eff2, X, T, cols, mask=(T[0].flatten()==1) & (X[0]['educ'] > -.1) & (X[0]['educ'] <.2))

In [ ]:
het_est2.fit_base()

In [ ]:
print(het_est2.policy_delta_complex())

In [ ]:
base_eff2 = het_est2.base_dynamic_effects(X['het'])

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree
tree = DecisionTreeRegressor(max_depth=3).fit(X['het'].values, base_eff2[0][:, 0])
plt.figure(figsize=(15, 10))
plot_tree(tree, feature_names=X['het'].columns)
plt.show()

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree
tree = DecisionTreeRegressor(max_depth=3).fit(X['het'].values, base_eff2[1][:, 0])
plt.figure(figsize=(15, 10))
plot_tree(tree, feature_names=X['het'].columns)
plt.show()

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree
tree = DecisionTreeRegressor(max_depth=3).fit(X['het'].values, base_eff2[1][:, 1])
plt.figure(figsize=(15, 10))
plot_tree(tree, feature_names=X['het'].columns)
plt.show()

In [ ]:
plot_true_v_est(base_eff2, X, T, cols, mask=(T[0].flatten()==1))

#### Validating heterogeneity finding on test set

In [ ]:
efftest = het_est2.dynamic_effects(Xtest['het'])

In [ ]:
import copy
newXtest = copy.deepcopy(Xtest)
newXtest['het'] = pd.DataFrame({'cate0': efftest[0][:, 0] - np.mean(efftest[0][:, 0]), 
                                'cate11': efftest[1][:, 0] - np.mean(efftest[1][:, 0]),
                                'cate10': efftest[1][:, 1] - np.mean(efftest[1][:, 1])},
                               index=Xtest['het'].index)
newXtest['het']

In [ ]:
from sklearn.tree import DecisionTreeRegressor
distill_vals = {}
distill = DecisionTreeRegressor(max_depth=1, min_samples_leaf=1000).fit(X['het'].values, eff2[0][:, 0])
distill_vals['cate0'] = distill.predict(newXtest[0].values[:, :X['het'].shape[1]])
distill = DecisionTreeRegressor(max_depth=1, min_samples_leaf=1000).fit(X['het'].values, eff2[1][:, 0])
distill_vals['cate11'] = distill.predict(newXtest[0].values[:, :X['het'].shape[1]])
distill = DecisionTreeRegressor(max_depth=1, min_samples_leaf=1000).fit(X['het'].values, eff2[1][:, 1])
distill_vals['cate10'] = distill.predict(newXtest[0].values[:, :X['het'].shape[1]])

In [ ]:
import copy
newXtest = copy.deepcopy(Xtest)
newXtest['het'] = pd.get_dummies(pd.DataFrame(distill_vals, index=Xtest['het'].index),
                                 columns=['cate0', 'cate10', 'cate11'], drop_first=True)
newXtest['het'].columns = ['cate0', 'cate10', 'cate11']
newXtest['het']

In [ ]:
plt.hist(np.prod(newXtest['het'], axis=1))
plt.show()

In [ ]:
import copy
from sklearn.tree import DecisionTreeRegressor, plot_tree
newXtest = copy.deepcopy(Xtest)
distill_vals = {}
distill = DecisionTreeRegressor(max_depth=2, min_samples_leaf=1000).fit(X['het'].values,
                                                                        np.hstack([eff2[0][:, [0]], eff2[1]]))
plot_tree(distill, feature_names=X['het'].columns)

In [ ]:
distill_vals['cate'] = distill.apply(newXtest['het'].values)

In [ ]:
distill.predict(newXtest['het'])

In [ ]:
import copy
newXtest = copy.deepcopy(Xtest)
newXtest['het'] = pd.get_dummies(pd.DataFrame(distill_vals, index=Xtest['het'].index),
                                 columns=['cate'], drop_first=False)
newXtest['het']

In [ ]:
newXtest[0] = pd.concat([newXtest[0], newXtest['het']], axis=1)
newXtest[0]

In [ ]:
from econml.utilities import cross_product

class SimpleHeteroBlipSpec(BlipSpec):

    def phi(self, t, X, T, Tt):
        if t==1:
            return np.hstack([Tt, cross_product(Tt, X['het'][['cate11']].values),
                              cross_product(Tt, T[t-1]),
                              cross_product(Tt, T[t-1], X['het'][['cate10']].values)])
        else:
            return np.hstack([Tt, cross_product(Tt, X['het'][['cate0']].values)])

    def phi_names(self, t):
        out = [f't[{x}]' for x in range(self.n_treatments[t])]
        if t==1:
            out += [f't[{x}]*x0[cate11]' for x in range(self.n_treatments[t])]
            out += [f't[{x}]*lagt[{y}]' for y in range(self.n_treatments[t-1]) for x in range(self.n_treatments[t])]
            out += [f't[{x}]*lagt[{y}]*x0[cate10]' for y in range(self.n_treatments[t])
                                                for x in range(self.n_treatments[t])]
        else:
            out += [f't[{x}]*x0[cate0]' for x in range(self.n_treatments[t])]
        return out

In [ ]:
bs = SimpleHeteroBlipSpec().fit(Xtest, Ttest)
phi = bs.phi
phi_names = bs.phi_names

In [ ]:
est_test = SNMMDynamicDML(m=m, phi=phi, phi_names_fn=phi_names,
                         model_reg_fn=lambda X, y: get_poly_model_reg(X, y, degree=1, interaction_only=False), #lambda X, y: get_model_reg(X, y, degrees=[1, 2]),
                         model_final_fn=lambda: LinearRegression(),
                         verbose=1)

In [ ]:
est_test.fit(newXtest, Ttest, ytest, pi)

In [ ]:
for t in range(m):
    print(f'Period {t} effects')
    with pd.option_context("precision", 3):
        display(est_test.param_summary(t).summary_frame(alpha=0.01))

In [ ]:
est_test.policy_delta_simple_

In [ ]:
# bs = SimpleBlipSpec().fit(X, T)
bs = BlipSpec(heterogeneity=False, lags=True, lag_heterogeneity=False).fit(X, T)
phi = bs.phi
phi_names = bs.phi_names

In [ ]:
het_est_test = HeteroSNMMDynamicDML(m=m, phi=phi, phi_names_fn=phi_names,
                                model_reg_fn=lambda X, y: get_poly_model_reg(X, y, degree=2, interaction_only=False), #lambda X, y: get_model_reg(X, y, degrees=[1, 2]),
                                model_final_fn=lambda: LinearModelFinal(StatsModelsLinearRegression(fit_intercept=False),
                                                                        lambda x: x,
                                                                        fit_cate_intercept=False))

In [ ]:
het_est_test.fit(newXtest, Ttest, ytest, pi)

In [ ]:
het_est_test.param_summary(0).summary_frame()

In [ ]:
het_est_test.param_summary(1).summary_frame()

In [ ]:
temp = SNMMDynamicDML(m=m, phi=phi, phi_names_fn=phi_names,
                      model_reg_fn=lambda X, y: get_poly_model_reg(X, y, degree=1, interaction_only=False),
                      model_final_fn=lambda: LinearRegression())

In [ ]:
temp.fit(newXtest, Ttest, ytest, pi)

In [ ]:
temp.param_summary(0).summary_frame()

In [ ]:
temp.param_summary(1).summary_frame()